# Matrix Multiplication

Speed up matrix multiplication in C with iterative performance optimizations.

**Matmul FLOPS:** $2n^3$

**Matmul Algorithm:** For $n \times n$ matrices $A$, $B$, $C$, where $n = 2^k$, calculate $C = AB$:

```python
for i in range(n):
    for j in range(n):
        for k in range(n):
            C[i][j] = A[i][k] * B[k][j]
```

## Version 1: Naive Loop

In [1]:
#include<stdio.h>
#include<stdlib.h>
#include<time.h>

#define n 1024
double A[n][n];
double B[n][n];
double C[n][n];

int main() {
  int i, j, k;
  double start, end;
  struct timespec t;

  for(i=0; i<n; i++) {
    for(j=0; j<n; j++) {
      A[i][j] = (double)rand() / (double)RAND_MAX;
      B[i][j] = (double)rand() / (double)RAND_MAX;
      C[i][j] = 0.0;
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  start = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  for(i=0; i<n; i++) {
    for(j=0; j<n; j++) {
      for(k=0; k<n; k++) {
        C[i][j] += A[i][k] * B[k][j];
      }
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  end = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  printf("Time: %f\n", end-start);
  return 0;
}

Time: 3.201055


## Version 2: Switch Inner Loops

In [2]:
#include<stdio.h>
#include<stdlib.h>
#include<time.h>

#define n 1024
double A[n][n];
double B[n][n];
double C[n][n];

int main() {
  int i, j, k;
  double start, end;
  struct timespec t;

  for(i=0; i<n; i++) {
    for(j=0; j<n; j++) {
      A[i][j] = (double)rand() / (double)RAND_MAX;
      B[i][j] = (double)rand() / (double)RAND_MAX;
      C[i][j] = 0.0;
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  start = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  for(i=0; i<n; i++) {
    for(k=0; k<n; k++) {     // loop switched
      for(j=0; j<n; j++) {   // loop switched
        C[i][j] += A[i][k] * B[k][j];
      }
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  end = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  printf("Time: %f\n", end-start);
  return 0;
}

Time: 1.176454


## Version 3: Compiler Optimization

In [5]:
//%cflags: -O2
// using O2-level compiler optimization
#include<stdio.h>
#include<stdlib.h>
#include<time.h>

#define n 1024
double A[n][n];
double B[n][n];
double C[n][n];

int main() {
  int i, j, k;
  double start, end;
  struct timespec t;

  for(i=0; i<n; i++) {
    for(j=0; j<n; j++) {
      A[i][j] = (double)rand() / (double)RAND_MAX;
      B[i][j] = (double)rand() / (double)RAND_MAX;
      C[i][j] = 0.0;
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  start = (double)t.tv_sec + (double)t.tv_nsec / 1e9;
    
  for(i=0; i<n; i++) {
    for(k=0; k<n; k++) {
      for(j=0; j<n; j++) {
        C[i][j] += A[i][k] * B[k][j];
      }
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  end = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  printf("Time: %f\n", end-start);
  return 0;
}

Time: 0.148221


## Parallelize Outer Loop

In [6]:
//%cflags: -O2 -Xclang -fopenmp -I/opt/homebrew/opt/libomp/include -L/opt/homebrew/opt/libomp/lib -lomp
// using openmp to parallelize outer loop
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
#include<omp.h>

#define n 1024
double A[n][n];
double B[n][n];
double C[n][n];

int main() {
  int i, j, k;
  double start, end;
  struct timespec t;

  for(i=0; i<n; i++) {
    for(j=0; j<n; j++) {
      A[i][j] = (double)rand() / (double)RAND_MAX;
      B[i][j] = (double)rand() / (double)RAND_MAX;
      C[i][j] = 0.0;
    }
  }
    
  clock_gettime(CLOCK_MONOTONIC, &t);
  start = (double)t.tv_sec + (double)t.tv_nsec / 1e9;
    
  #pragma omp parallel for private(j,k)
  for(i=0; i<n; i++) {   // parallelized outer loop
    for(k=0; k<n; k++) {
      for(j=0; j<n; j++) {
        C[i][j] += A[i][k] * B[k][j];
      }
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  end = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  printf("Time: %f\n", end-start);
  return 0;
}

Time: 0.035591


**Next:** tile L1 cache, recursively tile all caches, SIMD

Matmul on two 1024x1024 arrays of rands:
- Naive python: 83 sec
- Naive C: 3.10 sec (27x rel/abs speedup)
- Swap inner loops (`j <-> k`): 1.17 sec (3x rel speedup, 71x abs speedup)
- Run with `-O2` compiler optimization flag: 0.147 sec (8x rel speedup, 565x abs speedup)
- Parallelize outer loop `i` with OpenMP: 0.038 sec (4x rel speedup, 2184x abs speedup)

In [8]:
//%cflags: -O2 -DACCELERATE_NEW_LAPACK -framework Accelerate
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
#include<Accelerate/Accelerate.h>

#define n 1024
double A[n][n];
double B[n][n];
double C[n][n];

int main() {
  int i, j;
  double start, end;
  struct timespec t;

  for(i=0; i<n; i++) {
    for(j=0; j<n; j++) {
      A[i][j] = (double)rand() / (double)RAND_MAX;
      B[i][j] = (double)rand() / (double)RAND_MAX;
      C[i][j] = 0.0;
    }
  }

  clock_gettime(CLOCK_MONOTONIC, &t);
  start = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  cblas_dgemm(CblasRowMajor, CblasNoTrans, CblasNoTrans, n, n, n, 1.0, &A[0][0], n, &B[0][0], n, 0.0, &C[0][0], n);

  clock_gettime(CLOCK_MONOTONIC, &t);
  end = (double)t.tv_sec + (double)t.tv_nsec / 1e9;

  printf("Time: %f\n", end-start);
  return 0;
}

Time: 0.007104
